In [ ]:
#This checks only the malicious link in the csv with the email message body.

import pandas
import win32com.client
import imaplib
import email
from email.header import decode_header
import webbrowser
import os
import sys

col_list = ["url"]
df = pandas.read_csv('malicious.csv', usecols=col_list)

# Connect to inbox
username = 'tpmpuser123@gmail.com'
password = 'Tp@mpuser999'
mail_server = 'imap.gmail.com'
imap_server = imaplib.IMAP4_SSL(host=mail_server)
imap_server.login(username, password)
imap_server.select('INBOX')  # Default is `INBOX`

search_criteria = 'ALL'
charset = None  # All
respose_code, message_numbers_raw = imap_server.search(charset, search_criteria)
message_numbers = message_numbers_raw[0].split()

# Fetch full message based on the message numbers obtained from search
for message_number in message_numbers:
    response_code, message_data = imap_server.fetch(message_number, '(RFC822)')
    for response in message_data:
        #print(response)
        if isinstance(response, tuple):
            # parse a bytes email into a message object
            msg = email.message_from_bytes(response[1])
            # decode the email subject
            subject, encoding = decode_header(msg["Subject"])[0]
            if isinstance(subject, bytes):
                # if it's a bytes, decode to str
                subject = subject.decode(encoding)
            From, encoding = decode_header(msg.get("From"))[0]
            if isinstance(From, bytes):
                From = From.decode(encoding)
            # if the email message is multipart
            if msg.is_multipart():
                # iterate over email parts
                for part in msg.walk():
                    # extract content type of email
                    content_type = part.get_content_type()
                    content_disposition = str(part.get("Content-Disposition"))
                    try:
                        # get the email body
                        body = part.get_payload(decode=True).decode()
                        for link in df["url"]:
                            for letters in body.split():
                                if link == letters:
                                    imap_server.create('malicious')
                                    imap_server.store(message_number, '+X-GM-LABELS', 'malicious')
                                    imap_server.store(message_number, '+FLAGS', '\Deleted')  # Mark as read
                                    imap_server.store(message_number, '-FLAGS', '\Deleted')  # Mark as unread
                                    imap_server.expunge()
                                    print("Subject:", subject)
                                    print("From:", From)
                                    print("Found email with malicious link!")
                                    print("Malicious link:", letters)
                                    print("")
                                else:
                                    continue
                    except:
                        pass
imap_server.close()
imap_server.logout()

#This script runs the .csv file which contains all the malicious and phishing websites against the email message body by
#checking every words in the body. The, if there is a link that appears in the body and same as the csv file, it will
#move the email to the malicious folder/label and delete the mail from inbox and expunge it. This is a little bit buggy
#however, it works if there is a malicious link that can be found in the csv file. just need to run again.

In [ ]:
# This checks Received-SPF(Sender Policy Framework) and check malicious link with the csv to the message body.
#Excluding VirusTotal and Whois domain

import pandas
import win32com.client
import imaplib
import email
from email.header import decode_header
import webbrowser
import os
import sys
import whois
import requests
import time
import json

col_list = ["url"]
df = pandas.read_csv('malicious.csv', usecols=col_list)

# Connect to inbox
username = 'tpmpuser123@gmail.com'
password = 'Tp@mpuser999'
mail_server = 'imap.gmail.com'
imap_server = imaplib.IMAP4_SSL(host=mail_server)
imap_server.login(username, password)
imap_server.select('Inbox')  # Default is `INBOX`

search_criteria = 'ALL'
charset = None  # All
respose_code, message_numbers_raw = imap_server.search(charset, search_criteria)
message_numbers = message_numbers_raw[0].split()

# Fetch full message based on the message numbers obtained from search
for message_number in message_numbers:
    response_code, message_data = imap_server.fetch(message_number, '(RFC822)')
    for response in message_data:
        #print(response)
        if isinstance(response, tuple):
            # parse a bytes email into a message object
            msg = email.message_from_bytes(response[1])
            subject = msg.get("Subject", None)
            getFrom = msg.get("From", None)
            #getFrom = getFrom.split()
            #getFrom = getFrom[-1].strip('<>')
            returnPath = msg.get("Return-Path", None)
            returnPath = returnPath.strip('<>')
            receivedSPF = msg.get("Received-SPF", None).split()
            # if the email message is multipart
            if msg.is_multipart():
                # iterate over email parts
                for part in msg.walk():
                    # extract content type of email
                    content_type = part.get_content_type()
                    content_disposition = str(part.get("Content-Disposition"))
                    try:
                        # get the email body
                        body = part.get_payload(decode=True).decode()
                        for link in df["url"]:
                            for letters in body.split():
                                if link == letters:
                                    checkIP = whois.whois(link)
                                    test = checkIP.get("domain_name", None)
                                    imap_server.create('malicious')
                                    imap_server.store(message_number, '+X-GM-LABELS', 'malicious')
                                    imap_server.store(message_number, '+FLAGS', '\Deleted')  # read mail remove from inbox
                                    imap_server.store(message_number, '-FLAGS', '\Deleted')  # unread mail remove from inbox
                                    imap_server.expunge()
                                    print("Subject:", subject)
                                    print("From:", getFrom)
                                    print("Domain name maybe malicious:", test, ". Check with virustotal.com")
                                    print("Found email with malicious link!")
                                    print("Malicious link:", letters)
                                    print("")
                                elif receivedSPF[0] != "pass":
                                    imap_server.create('malicious')
                                    imap_server.store(message_number, '+X-GM-LABELS', 'malicious')
                                    imap_server.store(message_number, '+FLAGS', '\Deleted')  # read mail remove from inbox
                                    imap_server.store(message_number, '-FLAGS', '\Deleted')  # unread mail remove from inbox
                                    imap_server.expunge()
                                    print("Subject:", subject)
                                    print("From:", getFrom)
                                    print("SPF Status:", receivedSPF[0], ". Not secure.")
                                else:
                                    continue
                    except:
                        pass
                    
imap_server.close()
imap_server.logout()

#This script runs the .csv file which contains all the malicious and phishing websites against the email message body by
#checking every words in the body. The, if there is a link that appears in the body and same as the csv file, it will
#move the email to the malicious folder/label and delete the mail from inbox and expunge it. This is a little bit buggy
#however, it works if there is a malicious link that can be found in the csv file. just need to run again.

In [1]:
# This checks Received-SPF(Sender Policy Framework) and check malicious link with the csv to the message body.
# This includes VirusTotal API and Whois domain lookup

import pandas
import win32com.client
import imaplib
import email
from email.header import decode_header
import webbrowser
import os
import sys
import whois
import requests
import time
import json

col_list = ["url"]
df = pandas.read_csv('malicious.csv', usecols=col_list)

API_key = '9a620234276d322d185e00b59e25242ec06464b994c556d392d2ded861f2e9fe'
url = 'https://www.virustotal.com/vtapi/v2/url/report'
parameters = {'apikey': API_key, 'resource': df}
response= requests.get(url=url, params=parameters)
json_response= json.loads(response.text)

# Connect to inbox
username = 'tpmpuser123@gmail.com'
password = 'Tp@mpuser999'
mail_server = 'imap.gmail.com'
imap_server = imaplib.IMAP4_SSL(host=mail_server)
imap_server.login(username, password)
imap_server.select('Inbox')  # Default is `INBOX`

search_criteria = 'ALL'
charset = None  # All
respose_code, message_numbers_raw = imap_server.search(charset, search_criteria)
message_numbers = message_numbers_raw[0].split()

# Fetch full message based on the message numbers obtained from search
for message_number in message_numbers:
    response_code, message_data = imap_server.fetch(message_number, '(RFC822)')
    for response in message_data:
        #print(response)
        if isinstance(response, tuple):
            # parse a bytes email into a message object
            msg = email.message_from_bytes(response[1])
            subject = msg.get("Subject", None)
            getFrom = msg.get("From", None)
            #getFrom = getFrom.split()
            #getFrom = getFrom[-1].strip('<>')
            returnPath = msg.get("Return-Path", None)
            returnPath = returnPath.strip('<>')
            receivedSPF = msg.get("Received-SPF", None).split()
            # if the email message is multipart
            if msg.is_multipart():
                # iterate over email parts
                for part in msg.walk():
                    # extract content type of email
                    content_type = part.get_content_type()
                    content_disposition = str(part.get("Content-Disposition"))
                    try:
                        # get the email body
                        body = part.get_payload(decode=True).decode()
                        for link in df["url"]:
                            for letters in body.split():
                                if link == letters:
                                    parameters = {'apikey': API_key, 'resource': link} #VirusTotal only can check each link
                                    response= requests.get(url=url, params=parameters) # 4 times per minute!
                                    json_response = json.loads(response.text)
                                    if json_response['response_code'] <= 0:
                                        getstatus = "empty"
                                    elif json_response['response_code'] >= 1:
                                        if json_response['positives'] <= 0:
                                            getstatus = "positive"
                                        else:
                                            getstatus = "malicious"
                                            print(link)
                                            print("This link above is malicious according to VirusTotal!")
                                            print("")
                                    if getstatus == "malicious":
                                        checkIP = whois.whois(link)
                                        test = checkIP.get("domain_name", None)
                                        imap_server.create('malicious')
                                        imap_server.store(message_number, '+X-GM-LABELS', 'malicious')
                                        imap_server.store(message_number, '+FLAGS', '\Deleted')  # read mail remove from inbox
                                        imap_server.expunge()
                                        print("Subject:", subject)
                                        print("From:", getFrom)
                                        print("Domain name that is malicious:", test)
                                        print("Found email with malicious link!")
                                        print("Malicious link:", letters)
                                        print("")
                                elif receivedSPF[0] != "pass":
                                    imap_server.create('malicious')
                                    imap_server.store(message_number, '+X-GM-LABELS', 'malicious')
                                    imap_server.store(message_number, '+FLAGS', '\Deleted')  # read mail remove from inbox
                                    imap_server.expunge()
                                    print("Subject:", subject)
                                    print("From:", getFrom)
                                    print("SPF Status:", receivedSPF[0], ". Not secure.")
                                else:
                                    continue
                    except:
                        pass
                    
imap_server.close()
imap_server.logout()

#This script runs the .csv file which contains all the malicious and phishing websites against the email message body by
#checking every words in the body. The, if there is a link that appears in the body and same as the csv file, it will
#move the email to the malicious folder/label and delete the mail from inbox and expunge it. It uses VirusTotal to double check
#the link if it is malicious and uses Whois domain lookup to look for the domain. However, this is a little bit buggy, meaning
#it searches the email message body for malicious link but slim chance of missing it. In addition, if it found malicious link
#and scans with the VirusTotal, it sometimes will not move the email to the malicious label. Moreover, VirusTotal only allows
#us to scan 4 links every minute. Thus, it works if there is a malicious link that can be found in the csv file and VirusTotal.
# It just needs to run again and again. 

http://soscaboverde.org.cv/Citibank/
This link above is malicious according to VirusTotal!

https://inpost-pl.3ds-security.icu/1626790152271
This link above is malicious according to VirusTotal!

https://rakuten.card-2r.club/l
This link above is malicious according to VirusTotal!

Subject: tewfw
From: SO HONG YAO <1905616G@student.tp.edu.sg>
Domain name that is malicious: card-2r.club
Found email with malicious link!
Malicious link: https://rakuten.card-2r.club/l

https://leboncoin-prepaid.fr/id/id_paiement_rib.php
This link above is malicious according to VirusTotal!

Subject: malciious
From: SO HONG YAO <1905616G@student.tp.edu.sg>
Domain name that is malicious: leboncoin-prepaid.fr
Found email with malicious link!
Malicious link: https://leboncoin-prepaid.fr/id/id_paiement_rib.php



('BYE', [b'LOGOUT Requested'])